In [1]:
import random
from copy import deepcopy
import networkx as nx
from networkx.algorithms.community import kernighan_lin_bisection

In [2]:
class Grafo:
    
    def __init__(self):
        self.grafo = {}
    
    def import_m(self, M='Matriz.txt'):
        """
        Importa una matris de un documento de texto
        """
        with open(M, "r" , encoding="utf-8") as m:
            Matriz = m.read().split('\n')
        
        return Matriz
        
    def matrix_2_dict(self,data): 
        for linea in data:
            origen, destino, peso = linea.split()
            peso = int(peso)

            if origen not in self.grafo:
                self.grafo[origen] = {}
            if destino not in self.grafo:
                self.grafo[destino] = {}
    
            self.grafo[origen][destino] = peso
            self.grafo[destino][origen] = peso

        return self.grafo
    
    def vrp(self, grafo,nodo_c):
        
        G = nx.Graph(grafo)
        particiones = list(kernighan_lin_bisection(G))
        print(particiones)
        l =[]
        for particion in particiones:
            #print(type(particion))
            self.grafo = deepcopy(grafo)
            #print(self.grafo)
            if nodo_c in particion:
                particion.remove(nodo_c)

            for nodo in particion:
                if nodo in self.grafo:
                    del self.grafo[nodo]
                for nodo_conectado in self.grafo:
                    if nodo in self.grafo[nodo_conectado]:
                        del self.grafo[nodo_conectado][nodo]
            #print(self.grafo)

            l.append(self.grafo)
    
        return l
        
  

In [3]:
def aco(grafo, grafo_s):
    """
    Algoritmo de la colonia de hormigas
    
    Entrada : Un Diccionario que reprencenta un grafo, grafo incial y grafo que se busca
    
    Salida: una Ruta
    
    
    """
    
    
    global valores_divididos, Peso,N, T, T_1, suma_recorrido,trancado, explorados,suma_total, grafo_k

    K = 3 #Cantidad de hormigas
    A = 1 #Influencia de feromonas
    B = 1 #Influencia de heuristica
    p = 0 #

    T = {nodo: {vecino: 1 for vecino in vecinos} for nodo, vecinos in grafo.items()}#Grafo de feromonas inicial
    N = {nodo: {vecino: 1 / valor if valor != 0 else 0 for vecino, valor in vecinos.items()} for nodo, vecinos in grafo.items()}# Grafo inverso
    Peso = {nodo: {vecino: (T[nodo][vecino]**A) * (N[nodo][vecino] ** B) for vecino in T[nodo]} for nodo in T}# T*N 
    suma_total = {x: sum(Peso[x].values()) for x, i in Peso.items()}# De todos los T*N
    valores_divididos = {clave1: {clave2: (Peso[clave1][clave2] / suma_total[clave1]) for clave2 in Peso[clave1]} for clave1 in Peso}# Grafo con la probabilidad de que se vaya por esa ruta 
    T_1 = {nodo: {vecino: peso * (1-p) for vecino, peso in conexiones.items()} for nodo, conexiones in T.items()}#Grafo de feromonas QUE r
    
    print("THIS IS THE PESO",Peso,"\n")
    
    print("This is suma_total",suma_total,"\n")
    
    print("THIS IS THE valores_divididos",valores_divididos,"\n")
    

    while K > 0:
        frontera = valores_divididos[grafo_s]
        explorados = [grafo_s]
        p = 0 #evaporacion  
        grafo_k = list(grafo.keys())
        grafo_k.remove(grafo_s)
        while True:
            print(f'Frontera: {frontera}');print(f'Explorados: {explorados}')
            #Selecciona lo que esta en la frontera 
            select = list(frontera.keys()), list(frontera.values());select1 = random.choices(select[0], select[1], k=1)[0]
            print(f'El nodo que se seleccionó {select1}\n')

            if select1 not in explorados:
                frontera = valores_divididos[select1]
                explorados.append(select1)
                grafo_k.remove(select1)
                

            if len(grafo_k) == 0 :
                
                print(f'Explorados: {explorados}')
                explorados.append(grafo_s)
                suma_recorrido = 1/sum(grafo[explorados[i]][explorados[i+1]] for i in range(len(explorados) - 1))
                print(f"La suma del recorrido es {suma_recorrido}")
            
                
                T_1 = {nodo: {vecino: peso * (1-p) for vecino, peso in conexiones.items()} for nodo, conexiones in T_1.items()}
                    
                for i in range(len(explorados) - 1):
                    T_1[explorados[i]][explorados[i + 1]] += suma_recorrido
                
                Peso = {nodo: {vecino: (T_1[nodo][vecino]**A) * (N[nodo][vecino] ** B) for vecino in T[nodo]} for nodo in T}
                suma_total = {x: sum(Peso[x].values()) for x, i in Peso.items()}
                valores_divididos = {clave1: {clave2: (Peso[clave1][clave2] / suma_total[clave1]) for clave2 in Peso[clave1]} for clave1 in Peso} 
                print("THIS IS THE PESO",Peso,"\n")
    
                print("This is suma_total",suma_total,"\n")
    
                print("THIS IS THE valores_divididos",valores_divididos,"\n")
                print(f'Esta es la ruta {explorados}\n')
                    
                break
        
        print(f'Esta es otra hormiga\n')
        print(f'Feromonas:{T_1}\n')
        
        K -= 1
    return explorados


In [5]:
if __name__ == "__main__":
    t=Grafo()
    c=t.import_m()
    b=t.matrix_2_dict(c)
    n = t.vrp(b,"A")
    print(n)

[{'C', 'A'}, {'B', 'D'}]
[{'A': {'B': 2, 'D': 3}, 'B': {'A': 2, 'D': 4}, 'D': {'A': 3, 'B': 4}}, {'A': {'C': 7}, 'C': {'A': 7}}]


In [5]:
b

{'A': {'B': 2, 'C': 7, 'D': 3},
 'B': {'A': 2, 'C': 8, 'D': 4},
 'C': {'A': 7, 'B': 8, 'D': 9},
 'D': {'A': 3, 'B': 4, 'C': 9}}

In [6]:
aco(b,"A")

THIS IS THE PESO {'A': {'B': 0.5, 'C': 0.14285714285714285, 'D': 0.3333333333333333}, 'B': {'A': 0.5, 'C': 0.125, 'D': 0.25}, 'C': {'A': 0.14285714285714285, 'B': 0.125, 'D': 1.0}, 'D': {'A': 0.3333333333333333, 'B': 0.25, 'C': 1.0}} 

This is suma_total {'A': 0.976190476190476, 'B': 0.875, 'C': 1.2678571428571428, 'D': 1.5833333333333333} 

THIS IS THE valores_divididos {'A': {'B': 0.5121951219512196, 'C': 0.14634146341463417, 'D': 0.34146341463414637}, 'B': {'A': 0.5714285714285714, 'C': 0.14285714285714285, 'D': 0.2857142857142857}, 'C': {'A': 0.11267605633802817, 'B': 0.09859154929577466, 'D': 0.7887323943661972}, 'D': {'A': 0.21052631578947367, 'B': 0.15789473684210528, 'C': 0.6315789473684211}} 

Frontera: {'B': 0.5121951219512196, 'C': 0.14634146341463417, 'D': 0.34146341463414637}
Explorados: ['A']
El nodo que se seleccionó B

Frontera: {'A': 0.5714285714285714, 'C': 0.14285714285714285, 'D': 0.2857142857142857}
Explorados: ['A', 'B']
El nodo que se seleccionó A

Frontera: {'A'

['A', 'D', 'C', 'B', 'A']

In [9]:
print(carro,carro1)

['A', 'F', 'C', 'A'] ['A', 'B', 'E', 'D', 'A']


In [8]:
carro=aco(n[0],"A");carro1=aco(n[1],"A")

Frontera: {'C': 0.7142857142857143, 'F': 0.28571428571428575}
Explorados: ['A']
El nodo que se seleccionó C

Frontera: {'A': 0.6000000000000001, 'F': 0.4}
Explorados: ['A', 'C']
El nodo que se seleccionó A

Frontera: {'A': 0.6000000000000001, 'F': 0.4}
Explorados: ['A', 'C']
El nodo que se seleccionó A

Frontera: {'A': 0.6000000000000001, 'F': 0.4}
Explorados: ['A', 'C']
El nodo que se seleccionó A

Frontera: {'A': 0.6000000000000001, 'F': 0.4}
Explorados: ['A', 'C']
El nodo que se seleccionó F

Explorados: ['A', 'C', 'F']
La suma del recorrido es 0.1
Esta es la ruta ['A', 'C', 'F', 'A']

Esta es otra hormiga

Feromonas:{'A': {'C': 1.0, 'F': 0.9}, 'C': {'A': 0.9, 'F': 1.0}, 'F': {'A': 1.0, 'C': 0.9}}

Frontera: {'C': 0.7352941176470588, 'F': 0.2647058823529412}
Explorados: ['A']
El nodo que se seleccionó C

Frontera: {'A': 0.574468085106383, 'F': 0.425531914893617}
Explorados: ['A', 'C']
El nodo que se seleccionó A

Frontera: {'A': 0.574468085106383, 'F': 0.425531914893617}
Explorados:

['A', 'B', 'C', 'D', 'F', 'E', 'A']